# Python Write Program from MySQL to Excel

In [2]:
import mysql.connector
from openpyxl import Workbook

mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    password='151198',
    database='coffeedata',
    auth_plugin='mysql_native_password'
)

queries_and_sheets = [
    ("SELECT d.item_id itemId, c.item_name itemName, c.item_cat itemCategory, c.item_size itemSize, c.sku stockKeepingUnit, count(d.order_id) as totalOrdered, sum(d.quantity) as quantityOrderedAndSold, c.item_price itemPrice, c.item_price * sum(d.quantity) AS totalRevenue FROM coffeedata.items c INNER JOIN coffeedata.orders d ON c.item_id = d.item_id GROUP BY d.item_id, c.item_name, c.item_cat, c.item_size, c.sku, c.item_price ORDER BY quantityOrderedAndSold DESC, totalRevenue DESC;", "Query 1"),
    ("SELECT * , ROUND(f.itemPrice - f.productionCost, 3) AS profit, ROUND(((f. itemPrice - f.productionCost)/f.productionCost) * 100, 2) AS percentageProfit	FROM (SELECT c.item_id itemId, c.item_name itemName, e.recipe_id stockKeepingUnit, ROUND(SUM(e.quantity * a.unitPrice),3) AS productionCost, c.item_price itemPrice	FROM coffeedata.recipes e	INNER JOIN (SELECT a.*, ROUND (a.ing_price/a.ing_weight,3) as unitPrice	FROM coffeedata.ingredients a) a	ON e.ing_id = a.ing_id	INNER JOIN coffeedata.items c	ON e.recipe_id = c.sku	GROUP BY c.item_id, c.item_name, e.recipe_id, c.item_price) f	ORDER BY profit DESC;", "Query 2"),
    ("SELECT hour(z.created_at) AS orderHour, sum(z.quantity) AS sales, count(z.item_id) AS orders, Round(sum(y.productionCost * z.quantity), 2) AS productionCostOfOrderPerHour, Round(sum(y.Profit * z.quantity), 2) AS profitMadePerHour  FROM orders z	LEFT OUTER JOIN (SELECT f.item_id, f.item_name, f.recipe_id, f.productionCost, f.item_price, ROUND(f.item_price - f.productionCost, 3) AS profit	FROM (SELECT c.item_id, c.item_name, e.recipe_id, ROUND(SUM(e.quantity * a.unitPrice), 3) AS productionCost, c.item_price	FROM coffeedata.recipes e	INNER JOIN (SELECT a.*, ROUND(a.ing_price / a.ing_weight, 3) AS unitPrice	FROM coffeedata.ingredients a) a	ON e.ing_id = a.ing_id	INNER JOIN coffeedata.items c	ON e.recipe_id = c.sku	GROUP BY c.item_id, c.item_name, e.recipe_id, c.item_price) f	ORDER BY profit DESC) y on z.Item_Id = y.Item_id  GROUP BY orderHour","Query 3"),
    ("SELECT h.staff_id staffId, h.first_name firstName, h.last_name lastName, h.position, h.sal_per_hour salaryPerHour, g.day_of_week dayOfTheWeek, f.date, SUM(g.end_time - g.start_time) AS hoursWorked, sal_per_hour * SUM(g.end_time - g.start_time) AS dailySalary FROM coffeedata.staff h INNER JOIN coffeedata.rota f ON h.staff_id = f.staff_id INNER JOIN coffeedata.shift g ON f.shift_id = g.shift_id GROUP BY f.date,  g.day_of_week, h.staff_id, h.first_name, h.last_name, h.position, h.sal_per_hour ORDER BY f.staff_id","Query 4"),
    ("SELECT s.item_cat itemCategory, d.in_or_out dineInOrTakeOut, SUM(quantity) totalOrder, SUM(s.totalRevenue) AS totalRevenue FROM coffeedata.orders d INNER JOIN (SELECT d.item_id, c.item_name, c.item_cat, c.item_size, c.sku, count(d.order_id) as totalOrdered, sum(d.quantity) as quantityOrdered_and_sold, c.item_price, c.item_price * sum(d.quantity) AS totalRevenue FROM coffeedata.items c INNER JOIN coffeedata.orders d ON c.item_id = d.item_id GROUP BY d.item_id, c.item_name, c.item_cat, c.item_size, c.sku, c.item_price ORDER BY quantityOrdered_and_sold DESC, totalRevenue DESC) s ON d.item_id = s.item_id GROUP BY d.in_or_out, s.item_cat ORDER BY d.in_or_out DESC;","Query 5"),
    ("SELECT g.shift_id shiftId, f.date Date, g.start_time startTime, g.end_time endTime, COUNT(d.order_id) AS totalOrders FROM coffeedata.rota f RIGHT JOIN coffeedata.shift g ON f.shift_id = g.shift_id LEFT JOIN (SELECT d.order_id, date_format(d.created_at, '%Y-%m-%d') as Date FROM coffeedata.orders d) d ON f.Date = d.date GROUP BY g.shift_id, f.date, g.start_time, g.end_time ORDER BY totalOrders DESC, g.shift_id","Query 6")
]

wb = Workbook()
for query, sheet_name in queries_and_sheets:
    cursor = mydb.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = cursor.column_names
    
    if sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
    else:
        ws = wb.create_sheet(title=sheet_name)
    
    ws.append(columns)
    
    for row in rows:
        ws.append(row)
    
    cursor.close()

if 'Sheet' in wb.sheetnames and len(wb.sheetnames) > len(queries_and_sheets):
    del wb['Sheet']

sqlResult_file = 'sqlResult1.xlsx'
wb.save(sqlResult_file)

mydb.close()

print(f"Data has been written to {sqlResult_file}")

Data has been written to sqlResult1.xlsx
